In [1]:
from StroblSimFuns import *

lambdas = [1.0, 10.0]
relevance = 0.2
shrink_modes = ["hs_entropy"]
n_replications = 1
clf_type="rf"
n_samples=100
max_depth = 10

if clf_type == "rf":
    hsc = ShrinkageClassifier(RandomForestClassifier(max_depth=max_depth))
elif clf_type == "dt":
    hsc = ShrinkageClassifier(DecisionTreeClassifier(max_depth=max_depth))

param_grid = {"shrink_mode": [shrink_modes[0]], "lmb": lambdas}

/home/markusloecher/research/XAI/AugmentedHierarchicalShrinkage/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#!pip install shap
#!pip install numba
from StroblSimFuns import *

/home/markusloecher/research/XAI/AugmentedHierarchicalShrinkage/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
X, y = simulate_Strobl(n_samples, relevance)
X_test, y_test = simulate_Strobl(n_samples, relevance)

In [6]:
hsc = ShrinkageClassifier(RandomForestClassifier(max_depth=10), 
                          lmb=5, shrink_mode= "hs")
hsc.fit(X,y)
generate_SHAP(X, X_test, hsc.estimator_)

array([ 4.28181535, 17.73525773,  2.71316474,  3.27682935,  3.23443401])

In [5]:
lmb_scores = cross_val_shrinkage(
                hsc, X, y, param_grid, n_splits=5, score_fn = "AUC", 
                n_jobs=1, return_param_values=False)

In [7]:
best_idx = np.argmax(lmb_scores)
best_lmb = lambdas[best_idx]
best_lmb
hsc.set_shrink_params(shrink_mode=shrink_modes[0], lmb=best_lmb)

In [8]:
explainer = TreeExplainer(hsc.estimator_, X)
shap_values = np.array(explainer.shap_values(X_test))


In [9]:
shap_values.shape

(2, 100, 5)

In [11]:
np.sum(np.abs(shap_values[0,:,:]),axis=0)

array([ 2.33440967, 11.2550403 ,  1.84290559,  2.40542419,  2.87266079])

In [7]:
scores = joblib.load("output-2023-05-03-21-49/scores.pkl")
pars = joblib.load("output-2023-05-03-21-49/params.pkl")
bestlambdas = joblib.load("output-2023-05-03-21-49/bestlambdas.pkl")

In [22]:
relevances_str = list(scores.keys())
relevances_str
relevances = [float(i)/100 for i in relevances_str]
relevances

[0.0, 0.1, 0.2]

In [25]:
d = scores['00']["hs"].shape
d[0]

45

In [23]:
["{:.2f}".format(rel)[2:] for rel in relevances]

['00', '10', '20']

In [15]:
list(scores['00'].keys())

['hs', 'hs_entropy', 'hs_log_cardinality', 'hs_global_entropy']